In [2]:
import tkinter as tk
import pandas as pd
import pyaudio
import wave
import re

In [9]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 10
START_STRING = 0

In [10]:
flag = True
number = 0

In [11]:
def get_strings():
    strings = None
    with open('text.txt', 'r') as f:
        strings = f.read()
    strings = strings.split('. ')
    strings = [re.sub('[^A-Za-z0-9, \".\-\']+', '', i) + '.' for i in strings]
    df = pd.DataFrame(enumerate(strings), columns=['id', 'transcription'])
    df['id'] += 1
    return df

In [ ]:
def record_click():
    global flag, number
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    frames = []

    while flag:
        data = stream.read(CHUNK)
        frames.append(data)
        window.update()
    flag = True
    stream.close()
    wf = wave.open(f'Records/wavs/{number + 1}.wav', 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    number += 1
    text.delete('1.0', tk.END)
    text.insert(tk.END, df['transcription'][number])

In [ ]:
def rerecord_click():
    global number
    number -= 1
    text.delete('1.0', tk.END)
    text.insert(tk.END, df['transcription'][number])

In [ ]:
def stop_click():
    global flag
    flag = False

In [ ]:
p = pyaudio.PyAudio()

In [4]:
df = get_strings()

In [8]:
df.to_csv('Records/metadata.csv', index=False)

In [ ]:
window = tk.Tk()  
window.title("Record")
window.geometry('1000x300')
record = tk.Button(window, text='Start', command=record_click)
record.grid(column=0, row=0)
stop = tk.Button(window, text='Stop', command=stop_click)
stop.grid(column=0, row=1)
rerecord = tk.Button(window, text='Rerecord', command=rerecord_click)
rerecord.grid(column=0, row=2)
text = tk.Text(width=100, height=50)
text.insert(tk.END, df['transcription'][number])
text.grid(column=0, row=3)
window.mainloop()